In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import requests
import re
from sklearn.feature_extraction.text import CountVectorizer
from joblib import Parallel, delayed

In [9]:
#All URLs I would like to include:
breckinridge = 'https://www.airbnb.com/s/Breckenridge--CO/homes?adults=1&place_id=ChIJwecmbD32aocReqKAZn-PjWI&refinement_paths%5B%5D=%2Fhomes'
parkcity = 'https://www.airbnb.com/s/Park-City--UT/homes?adults=1&place_id=ChIJ_QNjLGMPUocRlFc3Jd_Ecdg&refinement_paths%5B%5D=%2Fhomes'
jacksonhole = 'https://www.airbnb.com/s/Jackson-Hole--WY/homes?adults=1&place_id=ChIJS3_P_FgaU1MRXIM6scsBHD0&refinement_paths%5B%5D=%2Fhomes'
vail = 'https://www.airbnb.com/s/Vail--CO/homes?adults=1&place_id=ChIJB89dUQVwaocRxKOafh_AzMk&refinement_paths%5B%5D=%2Fhomes'
steamboat = 'https://www.airbnb.com/s/Steamboat-Springs--CO/homes?adults=1&place_id=ChIJYUZWCYF7QocRfc9uSNGjqBs&refinement_paths%5B%5D=%2Fhomes'
bigsky = 'https://www.airbnb.com/s/Big-Sky--MT/homes?adults=1&place_id=ChIJNSw3_WUOUFMRyiuLqjtx-JU&refinement_paths%5B%5D=%2Fhomes'
telluride = 'https://www.airbnb.com/s/Telluride--CO/homes?adults=1&place_id=ChIJc_TmcHvYPocR4eO6cSF37jg&refinement_paths%5B%5D=%2Fhomes'
aspen = 'https://www.airbnb.com/s/Aspen--CO/homes?adults=1&place_id=ChIJfTxB93w5QIcRcvYseNxCK8E&refinement_paths%5B%5D=%2Fhomes'
tahoe = 'https://www.airbnb.com/s/Lake-Tahoe/homes?adults=1&place_id=ChIJUREfuaF4mYARILWv7q8fP4w&refinement_paths%5B%5D=%2Fhomes'
taos = 'https://www.airbnb.com/s/Taos--NM/homes?adults=1&place_id=ChIJsfwRf9pkF4cRgrepYYOR6pA&refinement_paths%5B%5D=%2Fhomes'

In [12]:
url_lst = [breckinridge, parkcity, jacksonhole, vail, steamboat, bigsky, telluride, aspen, tahoe, taos]

In [27]:
for url in url_lst:
    soupPage = getPage(url)
    listing_lst = getRoomClasses(soupPage)
    for listing in listing_lst:
        getListingLink(listing)

In [50]:
def getPage(url):
    result = requests.get(url)
    content = result.content
    BeautifulSoup(content, features = "lxml")
    return BeautifulSoup(content, features = "lxml")

In [5]:
#Information on different listings is shown with an image and information under the class '_8ssblpx'
def getRoomClasses(soupPage):
    rooms = soupPage.findAll("div", {"class": "_8ssblpx"})
    result = []
    for room in rooms:
        result.append(room)
    return result

In [21]:
def getListingLink(listing):
    return "http://airbnb.com" + listing.find("a")["href"]

In [28]:
def getListingTitle(listing):
    return listing.find("meta")["content"]

In [29]:
def getTopRow(listing):
    return listing.find("div", {"class": "_167qordg"}).text

In [30]:
def getRoomInfo(listing):
    return listing.find("div", {"class": "_kqh46o"}).text

In [31]:
def getBasicFacilities(listing):
    try:
        output = listing.findAll("div", {"class": "_kqh46o"})[1].text.replace(" ","")
    except:
        output = []
    return output

In [32]:
def getListingPrice(listing):
    return listing.find("div", {"class": "1fwiw8gv"}).text

In [33]:
def getListingRating(listing):
    return listing.find("span", {"class": "_krjbj"}).text

In [36]:
def getListingReviewNumber(listing):
    try:
        output = listing.findAll("span", {"class": "_krjbj"})[1].text
    except:
        output = -1
    return output

In [37]:
def extractInformation(soupPage):
    listings = getRoomClasses(soupPage)
    titles, links, toprows, roominfo, basicfacilities, prices, ratings, reviews = [], [], [], [], [], [], [], []
    for listing in listings:
        titles.append(getListingTitle(listing))
        links.append(getListingLink(listing))
        toprows.append(getListingLink(listing))
        roominfo.append(getListingLink(listing))
        basicfacilities.append(getListingLink(listing))
        prices.append(getListingLink(listing))
        ratings.append(getListingLink(listing))
        reviews.append(getListingLink(listing))
    dictionary = {"title": titles, "toprow": toprows, "roominfo": roominfo, "facilities": basicfacilities, "price": prices, "rating": ratings, "link": links, "review": reviews}
    return pd.DataFrame(dictionary)

In [38]:
def findNextPage(soupPage):
    try:
        nextpage = "https://airbnb.com" + soupPage.find("li", {"class": "_i66xk8d"}).find("a")["href"]
    except:
        nextpage = "no next page"
    return nextpage

In [39]:
def getPages(url):
    result = []
    while url != "no next page":
        page = getPage(url)
        result = result + [page]
        url = findNextPage(page)
    return result

In [40]:
def extractPages(url):
    pages = getPages(url)
    df = extractInformation(pages[0])
    for pagenumber in range(1, len(pages)):
        df = df.append(extractInformation(pages[pagenumber]))
    return df

In [41]:
url_lst = [["Breckinridge",breckinridge], 
           ["Park City", parkcity],
           ["Jacksonhole", jacksonhole],
           ["Vail", vail], 
           ["Steamboat Springs", steamboat],
           ["Big Sky", bigsky],
           ["Telluride", telluride],
           ["Aspen", aspen],
           ["Lake Tahoe", tahoe],
           ["Taos", taos]]

In [45]:
def scrapeURLs(listofURLs):
    print(listofURLs[0][0])
    df - extractPages(listofURLs[0][1])
    df.loc[:, "city"] = listofURLs[0][0]
    for i in range(1, len(listofURLs)):
        print(listofURLs[i][0])
        newrows = extractPages(listofURLs[i][1])
        newrows.loc[:, "city"] = listofURLs[i][0]
        df = df.append(newrows)
    return df

In [46]:
def getDescription(detailpage):
    return detailpage.find("div",{"class": "_eeq7h0"}).text

In [47]:
def getDetailedScores(detailpage):
    output = []
    scores = detailpage.findAll(class_ = '_a3qxec')
    try:
        for i in range(0, 6):
            split = scores[i].text.split(".")
            output.append(float(split[0][-1] + "." + split[1]))
    except:
        pass
    return output

In [48]:
def getHostInfo(detailpage):
    return detailpage.find(class_ = "_f47qa6").text

In [49]:
def setupDriver(url, waiting_time = 2.5):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(waiting_time)
    return driver

In [51]:
def getJSpage(url):
    driver = setupDriver(url)
    read_more_buttons = driver.find_elements_by_class_name("_1d079j1e")
    try:
        for i in range(2, len(read_more_buttons)):
            read_more_buttons[i].click()
    except:
        pass
    html = driver.page_source
    driver.close()
    return BeautifulSoup(html, features = "lxml")

In [52]:
def getAmenitiesPage(detailpage):
    
    link = detailpage.find(class_ = "_1v4ygly5")["href"]
    driver = setupDriver("https://airbnb.com" + link, 5)
    html = driver.page_source
    driver.close()
    return BeautifulSoup(html, features = "lxml")

In [53]:
first = True
scraped = 0
def getAddis(url):
    global first
    global scraped
    output = pd.DataFrame(columns=["details_page", "amenities_page", "link"])
    try:
        dp = getJSpage(url)
        output.loc[0] = [dp, getAmenitiesPage(dp), url]
    except:
        output.loc[0] = [-1, -1, url]
    if first:
        output.to_csv('intermediate_results_par.csv', mode='a', header=True, index=False)
        first = False
    else:
        output.to_csv('intermediate_results_par.csv', mode='a', header=False, index=False)
    scraped += 1
    print("Scraped: {}".format(scraped))

In [54]:
def getReviews(detailpage):
    reviews = detailpage.findAll(class_ = "_50mnu4")
    output = ""
    for review in reviews:
        output += review.text + "**-**"
    return output

In [55]:
def getAmenities(amenitiespage):
    amenities = amenitiespage.findAll(class_ = "_vzrbjl")
    output = ""
    for amenity in amenities:
        output += re.findall('[A-Z][^A-Z]*', amenity.text)[0] + "**-**"
    return output

In [56]:
def getResponseInfo(detailpage):
    try:
        output = detailpage.find(class_ = "_jofnfy").text
    except:
        output = ""
    return output

In [57]:
def cleanFacilities(df):
    df.loc[:, "facilities"] = df["facilities"].astype(str).str.replace("[","").str.replace("]","")
    vectorizer = CountVectorizer(decode_error = "ignore")
    X = vectorizer.fit_transform(df.facilities)
    bag_of_words = pd.DataFrame(X.toarray(), columns=vectorizer.get_features_names())
    return pd.concat([df.reset_index(drop=True).drop("facilities", axis=1), bag_of_words], axis=1)

In [58]:
def cleanTitle(df):
    df.loc[:, "name"] = df["title"].str.split(" null ", n=0, expand=True)[0].str.replace("-", "")
    df.loc[:, "location"] = df["title"].str.split(" null ", n=0, expand=True)[1].str.replace("-","").str.strip()
    return df.drop("title", axis=1)

In [59]:
def cleanTopRow(df):
    df.loc[:, 'roomtype'] = df["toprow"].str.split(" in ", n=0, expand=True)[0]
    df.loc[:, 'detailed_location'] = df["toprow"].str.split(" in ", n=0, expand=True)[1]
    return df.drop("toprow", axis=1)

In [ ]:
def cleanRoomInfo(df):
    df.loc[:, "guests"] = df.loc[:, "roominfo"].str.split(" . ", n=0, expand=True)[0].str.replace("guests", "")
    df.loc